<a href="https://colab.research.google.com/github/Mary82/cross-Modal-Learning/blob/master/ZSL_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,Input
from keras.layers.merge import Concatenate
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
import keras.backend as K
from keras.models import Model
from keras.layers.core import Lambda
from keras.optimizers import Adam
import random
import itertools
from keras.models import model_from_json
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/My Drive/Colab Notebooks/')
from ZSL_Utility import *

Using TensorFlow backend.


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
num_classes = 100
json_file = open('Models/cifar_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
cifar_model = model_from_json(loaded_model_json)
# load weights into new model
cifar_model.load_weights('Models/CIFAR100_Weight.h5')
print("CIFAR model loaded from disk")
cifar_model.summary()

CIFAR model loaded from disk
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 128)       3584      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 128)       147584    
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 128)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 256)       295168    
_________________________________________________________________
activation_3 (Activation)    (None, 15, 15, 256

In [3]:
img_emb_json_file = open('Models/img_emb_model.json', 'r')
loaded_model_json = img_emb_json_file.read()
img_emb_json_file.close()
img_emb_model = model_from_json(loaded_model_json)
# load weights into new model
img_emb_model.load_weights('Models/img_emb_model.h5')
print("Image embedding model loaded from disk")


Image embedding model loaded from disk


In [4]:
X_cnn_layer, X_emb, Ys = load_data(cifar_model, 'max_pooling2d_2', img_emb_model, num_classes)


169017344/169001437 [==============================] - 55s 0us/step


In [6]:
print  X_cnn_layer.shape
print X_emb.shape

(10000, 6, 6, 256)
(10000, 128)


In [0]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo)
    return dict




In [0]:
label_names = unpickle("Models/meta")
names = [x.replace('_', ' ') for x in label_names['fine_label_names']]
#print names
names[names.index('lawn mower')] = 'mower'

In [0]:
with open('Models/glove.6B.300d.txt', 'r') as f:
    word_emb_dict = {}
    for line in f :
        split_line = line.split()
        word = split_line[0]
        vector = np.array(split_line[1:])
        word_emb_dict[word] = vector
        

In [13]:
subjects = np.unique(Ys).tolist()
final_label = []
for subj in subjects:
    if names[subj] in word_emb_dict :
        final_label.append(subj)
        
print "Availabe labels: ", len(final_label)

Availabe labels:  92


(1200, 1)

In [0]:
train_sub = random.sample( final_label,82) 
test_sub = list(set(final_label) - set(train_sub))
train_data = np.where(np.isin(Ys, train_sub))[0] 
test_data = np.where(np.isin(Ys, test_sub))[0] 

X_cnn_layer_train = X_cnn_layer[train_data]
X_emb_train = X_emb[train_data]
Y_train = Ys[train_data]

X_cnn_layer_test = X_cnn_layer[test_data]
X_emb_test = X_emb[test_data]
Y_test = Ys[test_data]


In [0]:
Y_labels_train = np.array(names)[Y_train]
Y_glove_train = [word_emb_dict[x[0]] for x in Y_labels_train]
Y_glove_train = np.array(Y_glove_train).astype('float64')

Y_labels_test = np.array(names)[Y_test]
Y_glove_test = [word_emb_dict[x[0]] for x in Y_labels_test]
Y_glove_test = np.array(Y_glove_test).astype('float64')


In [19]:
print Y_glove_train.shape
print Y_glove_test.shape

(8200, 300)
(1000, 300)


In [0]:
def build_model(embed_n_in, conv_n_in):
    emb_input = Input(embed_n_in)
    conv_input = Input(conv_n_in)
    X = Conv2D(128, (1,1), activation='elu')(conv_input)
    #X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(.3)(X)
    X = Conv2D(64, (1,1), activation='elu')(X)
    X = Flatten()(X)
    X = Dense(512, activation = 'elu')(X)
    X = Lambda(lambda  x: K.l2_normalize(x,axis=1))(X)
    X = Concatenate(axis = -1)([X,emb_input])
    X = Dropout(.3)(X)
    #X = Dense(600, activation = 'elu')(X)
    #X = Dropout(.2)(X)
    X = Dense(300,activation = 'elu' )(X)
    X = Lambda(lambda  x: K.l2_normalize(x,axis=1))(X)
    model = Model(inputs = [emb_input,conv_input], outputs = X)
    #model.load_weights('Models/ZSL_model.h5')
    return model
    


In [26]:
ZSL_model = build_model(X_emb_train.shape[1:], X_cnn_layer_train.shape[1:])
ZSL_model.summary()
#ZSL_model.load_weights('Models/ZSL_model.h5')


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 6, 6, 256)    0                                            
__________________________________________________________________________________________________
conv2d_5 (Conv2D)               (None, 6, 6, 128)    32896       input_6[0][0]                    
__________________________________________________________________________________________________
dropout_5 (Dropout)             (None, 6, 6, 128)    0           conv2d_5[0][0]                   
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 6, 6, 64)     8256        dropout_5[0][0]                  
__________________________________________________________________________________________________
flatten_3 

In [0]:
opt = keras.optimizers.sgd(lr = .0001)
ZSL_model.compile(loss=euclidean_distance_loss, optimizer= opt)


In [41]:
ZSL_model.fit([X_emb_train,X_cnn_layer_train], Y_glove_train, batch_size = 128, verbose = 1, epochs = 10)
ZSL_model.fit([X_emb_train,X_cnn_layer_train], Y_glove_train, batch_size = 128, verbose = 0, epochs = 3600)
ZSL_model.fit([X_emb_train,X_cnn_layer_train], Y_glove_train, batch_size = 128, verbose = 1, epochs = 10)
ZSL_model.save_weights('Models/ZSL_model_1.h5') 

Epoch 1/10
8200/8200 [==============================] - 1s 144us/step - loss: 5.9301
Epoch 2/10
8200/8200 [==============================] - 1s 92us/step - loss: 5.9250
Epoch 3/10
8200/8200 [==============================] - 1s 93us/step - loss: 5.9293
Epoch 4/10
8200/8200 [==============================] - 1s 93us/step - loss: 5.9285
Epoch 5/10
8200/8200 [==============================] - 1s 91us/step - loss: 5.9289
Epoch 6/10
8200/8200 [==============================] - 1s 91us/step - loss: 5.9283
Epoch 7/10
8200/8200 [==============================] - 1s 93us/step - loss: 5.9272
Epoch 8/10
8200/8200 [==============================] - 1s 90us/step - loss: 5.9286
Epoch 9/10
8200/8200 [==============================] - 1s 92us/step - loss: 5.9295
Epoch 10/10
8200/8200 [==============================] - 1s 92us/step - loss: 5.9289
Epoch 1/10
8200/8200 [==============================] - 1s 89us/step - loss: 5.9213
Epoch 2/10
8200/8200 [==============================] - 1s 89us/step - los

1000/1000 [==============================] - 0s 61us/step


6.886125499725342